In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import sys
import datetime as dt
from arch import arch_model

sys.path.insert(0, r'c:\Users\joneh\master_thesis\src')
from main_utils import *
from db_utils import *

In [ ]:
df = pd.read_csv(r'C:\Users\joneh\master_thesis\data\time_series\CLc1.csv')
# drop count
df.drop(columns=['COUNT', 'HIGH', 'LOW', 'OPEN'], inplace=True)

df.index = pd.to_datetime(df['Date'])

# calculate log returns

# check for negative CLOSE values
# if df['CLOSE'].min() < 0:

df['LOGRET'] = np.log(df['CLOSE'].apply(lambda x: complex(x, 0))).diff().apply(lambda x: x.real)

# else:
#     df['LOGRET'] = np.log(df['CLOSE']).diff()


# add GARCH(1,1)
am = arch_model(df['LOGRET'][1:] * 100, vol='Garch', p=1, o=0, q=1, dist='Normal')
res = am.fit(update_freq=5)

# add conditional volatility
df['GARCH'] = res.conditional_volatility / 100

display(df)

In [ ]:
df['YEAR'] = df.index.year
df['MONTH'] = df.index.month
df['DAY'] = df.index.day
df['WEEKDAY'] = df.index.weekday
df['WEEK'] = df.index.week

display(df)

### Save data

In [ ]:
# Enter filename here:
file_name = 'CLc1_processed.csv'
# Enter relative path for saving the file:
relative_path = 'data/time_series/'

df.to_csv(save_path(relative_path, file_name), index=True)